In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import scipy.signal
import scipy.stats

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'damselfly/data/datasets')
SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')

"""
Date: 6/25/2021
Description: template
"""


def shift_and_sum(signal, x_range, y_range, N):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    x_antenna = x_antenna.reshape((1, x_antenna.size)).repeat(N ** 2, axis = 0)
    y_antenna = y_antenna.reshape((1, y_antenna.size)).repeat(N ** 2, axis = 0)
    
    x_array = np.linspace(-x_range, x_range, N)
    y_array = np.linspace(-y_range, y_range, N)
    
    x_grid, y_grid = np.meshgrid(x_array, y_array)
    x_grid = x_grid.flatten().reshape((x_grid.size, 1)).repeat(nch, axis=-1)
    y_grid = y_grid.flatten().reshape((y_grid.size, 1)).repeat(nch, axis=-1)
    
    #print(x_antenna.shape, x_grid.shape)
    
    phase_grid = 2 * np.pi * np.sqrt((x_grid - x_antenna) ** 2 + (y_grid - y_antenna) ** 2) / wavelength_lo
    
    anti_spiral = angles.reshape((1, angles.size)).repeat(N**2, axis=0)
    
    summed_signals = np.matmul(np.exp(1j * (phase_grid-anti_spiral)), signal)
    
    summed_signals = summed_signals.reshape((N, N, summed_signals.shape[-1]))
    
    return summed_signals


def shift_signal(signal, x_new, y_new):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    x_antenna = x_antenna.reshape((x_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    y_antenna = y_antenna.reshape((y_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    
    d_old = np.sqrt(x_antenna ** 2 + y_antenna ** 2)
    d_new = np.sqrt((x_new - x_antenna) ** 2 + (y_new - y_antenna) ** 2)
    
    
    
    phase_shift = 2 * np.pi * (d_new - d_old) / wavelength_lo
    #print(phase_shift)
    
    shifted_signal = np.exp(-1j * phase_shift) * signal
    
    return shifted_signal
    
    
def shift_signal_time(signal, x_new, y_new, freq):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    Nsample = 8192
    fsample = 200e6
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    
    x_antenna = x_antenna.reshape((x_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    y_antenna = y_antenna.reshape((y_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    
    source_angle = -1 * 2 * np.pi * np.arange(0, Nsample, 1) * freq / fsample
    
    r_new = np.sqrt(x_new ** 2 + y_new ** 2)
    
    x_new = r_new * np.cos(source_angle).reshape((1, Nsample)).repeat(nch, axis=0)
    y_new = r_new * np.sin(source_angle).reshape((1, Nsample)).repeat(nch, axis=0)
    
    d_old = np.sqrt(x_antenna ** 2 + y_antenna ** 2)
    d_new = np.sqrt((x_new - x_antenna) ** 2 + (y_new - y_antenna) ** 2)
    
    phase_shift = 2 * np.pi * (d_new - d_old) / wavelength_lo
    print(phase_shift.shape)
    
    shifted_signal = np.exp(-1j * phase_shift) * signal
    
    return shifted_signal
    
    
    

In [ ]:
#shift_and_sum(np.zeros((60, 8192)), 0.03, 0.03, 51)

In [ ]:
os.listdir(os.path.join(PATH, 'mayfly/data/datasets'))

In [ ]:
os.listdir(os.path.join(PATH, 'damselfly/data/sim_data'))

# load data

In [ ]:
data = os.path.join(os.path.join(PATH, 'mayfly/data/datasets'), '211002_mf_84_100_slice8192.h5')
#data = os.path.join(os.path.join(PATH, 'damselfly/data/sim_data'), '210729_df_84_1slice.h5')

h5datafile = h5py.File(data, 'r')

energies = h5datafile['meta']['energy'][:]

select_energies = np.argwhere(abs(energies - 18550) <= 0.5).squeeze()

In [ ]:
data = os.path.join(os.path.join(PATH, 'mayfly/data/datasets'), '211002_mf_84_100_slice8192.h5')
#data = os.path.join(os.path.join(PATH, 'damselfly/data/sim_data'), '210729_df_84_1slice.h5')

h5datafile = h5py.File(data, 'r')

#print(h5datafile['data'].shape)

Nsignal = h5datafile['data'].shape[0]

#Nsignal = len(list(h5datafile['signal'].keys()))
Nsample = 8192
Nch = 60

dataset = np.zeros((select_energies.size, Nch * Nsample), dtype=np.complex64)
print(dataset.shape)

for i, index in enumerate(select_energies):
#    dataset[i, :] = np.fft.fftshift(np.fft.fft(h5datafile['data'][i, :].reshape(60, 8192).sum(axis=0))) / 8192
#    dataset[i, :] = h5datafile['signal'][f'{i}'][0, :]
    dataset[i, :] = (np.fft.fftshift((np.fft.fft(h5datafile['data'][index, :].reshape(60, 8192), axis=-1)), axes=-1) / 8192).flatten()

# beamforming

In [ ]:
signal = dataset[0, :].reshape((60, 8192))

summed_signal = shift_and_sum(signal, 0.03, 0.03, 101)

In [ ]:
plt.imshow(abs(summed_signal).sum(axis=-1) ** 2)

# augment position

In [ ]:
signal = dataset[0, :].reshape((60, 8192))

shifted_signal = shift_signal(signal, 0.001, 0.0)

summed_shifted_signal = shift_and_sum(shifted_signal, 0.03, 0.03, 101)

In [ ]:
plt.imshow(abs(summed_shifted_signal).sum(axis=-1) ** 2)

# augment position time

In [ ]:
data = os.path.join(os.path.join(PATH, 'mayfly/data/datasets'), '211002_mf_84_100_slice8192.h5')
#data = os.path.join(os.path.join(PATH, 'damselfly/data/sim_data'), '210729_df_84_1slice.h5')

h5datafile = h5py.File(data, 'r')

#print(h5datafile['data'].shape)

Nsignal = h5datafile['data'].shape[0]

#Nsignal = len(list(h5datafile['signal'].keys()))
Nsample = 8192
Nch = 60

dataset = np.zeros((select_energies.size, Nch * Nsample), dtype=np.complex64)
print(dataset.shape)

for i, index in enumerate(select_energies):
#    dataset[i, :] = np.fft.fftshift(np.fft.fft(h5datafile['data'][i, :].reshape(60, 8192).sum(axis=0))) / 8192
#    dataset[i, :] = h5datafile['signal'][f'{i}'][0, :]
    dataset[i, :] = (h5datafile['data'][index, :])

In [ ]:
signal = dataset[0, :].reshape((60, 8192))

shifted_signal = shift_signal_time(signal, 0.02, 0.0, 5e3)

summed_shifted_signal = shift_and_sum(shifted_signal, 0.03, 0.03, 101)

In [ ]:
plt.imshow(abs(summed_shifted_signal).sum(axis=-1) ** 2)

In [ ]:
transpose_matrix = np.matmul(dataset, dataset.T)

evals, evecs = np.linalg.eig(transpose_matrix)

In [ ]:
print(evecs[:, 1].shape, dataset.T.shape)

In [ ]:
dataset_evec = np.matmul(dataset.T, evecs[:, 1].reshape((305, 1)))

In [ ]:
plt.plot(abs(dataset_evec))
plt.xlim(0, 2 * 8192)

In [ ]:
plt.plot(dataset_evec.real)
plt.plot(dataset_evec.imag)
plt.xlim(0, 8192)

In [ ]:
plt.plot(dataset_evec.real)
plt.plot(dataset_evec.imag)
plt.xlim(491520 - 14 * 8192, 491520 - 15 * 8192)

# select pc's

In [ ]:
pc_max_vals = abs(pca_range_evecs).max(axis=1)

In [ ]:
selected_pc_inds = np.argwhere(pc_max_vals >= 0.75)

select_pc = pca_range_evecs[selected_pc_inds[:, 0], :, selected_pc_inds[:, 1]]

print(select_pc.shape)


In [ ]:
n = 2
m = 3

ch = 1

var = 1.38e-23 * 10 * 200e6 * 50
#print(var / (np.sqrt(8192)))
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 8192)

noise = np.fft.fft(noise[:, 0] + 1j * noise[:, 1]) / 8192
pc = select_pc[n, :]
signal = dataset[m, ch, :]

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

#ax.plot(abs(dataset[m, :]))
#ax.plot(abs(noise))

#ax.plot(abs(scipy.signal.convolve(dataset[m, :], dataset[m, :], mode='full')))
#ax.plot(abs(scipy.signal.correlate(pc * np.sqrt((abs(dataset[m, ch, :]) ** 2 ).sum()), signal, mode='full')))


#ax.plot(abs(scipy.signal.convolve(noise, noise, mode='full')))

#fig = plt.figure(figsize=(8,5))
#ax = fig.add_subplot(1,1,1)

#norm_noise = noise / np.sqrt((abs(noise) ** 2).sum())
#norm_signal = dataset[m, :] / np.sqrt((abs(dataset[m, ch, :]) ** 2 ).sum())

ax.plot(abs(scipy.signal.convolve(pc * np.sqrt((abs(dataset[m, ch, :]) ** 2 ).sum()), np.flip(signal), mode='full')))
ax.plot(abs(scipy.signal.convolve(pc * np.sqrt((abs(dataset[m, ch, :]) ** 2 ).sum()), np.exp(-1j * 0 * np.pi / 4) * np.flip(signal), mode='full')))
#ax.plot(abs(scipy.signal.convolve(signal, signal, mode='full')))

# do pc projection then sum, compare to sum before projection

In [ ]:
n = 2
m = 3


var = 1.38e-23 * 10 * 200e6 * 50
#print(var / (np.sqrt(8192)))
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 8192)

noise = np.fft.fft(noise[:, 0] + 1j * noise[:, 1]) / 8192
pc = select_pc[n, :]

signal = dataset[m, :, :]

sum_signal = signal.sum(axis=0)
print

sum_proj_signal = abs(scipy.signal.fftconvolve(pc , np.flip(sum_signal), mode='full', axes=-1))

proj_signal = abs(scipy.signal.fftconvolve(pc.reshape((1, pc.size)).repeat(60, axis=0) , np.flip(signal, axis=-1), mode='full', axes=-1))


fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

ax.plot(sum_signal.real)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

ax.plot(signal[0, :].real)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

ax.plot(proj_signal[0, :])
#ax.plot(abs(noise))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

ax.plot(proj_signal.sum(axis=0))
ax.plot(sum_proj_signal)

# example multi-component projection with noise

In [ ]:
#N_comp = 256
m = 0
projection_n, projection_noise, projection_signal = [], [], []
signal = dataset[m, :, :]

for n in range(select_pc.shape[0]):
    
    var = 1.38e-23 * 10 * 200e6 * 50
    #print(var / (np.sqrt(8192)))
    noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 8192 * 60)
    noise = np.fft.fft(((noise[:, 0] + 1j * noise[:, 1]) / 8192).reshape((60, 8192)), axis=-1)
    
    pc = select_pc[n, :]
    
    proj_signal = abs(scipy.signal.fftconvolve(pc.reshape((1, pc.size)).repeat(60, axis=0) , noise + np.flip(signal, axis=-1), mode='full', axes=-1))
    proj_noise = abs(scipy.signal.fftconvolve(pc.reshape((1, pc.size)).repeat(60, axis=0) , noise , mode='full', axes=-1))
    
    projection_n.append(proj_signal.sum(axis=0).max() )
    projection_noise.append(proj_noise.sum(axis=0).max())
    
    
projection_n = np.array(projection_n)
projection_noise = np.array(projection_noise)
#projection_signal = np.array(projection_signal)

#print(projection_n.mean(), projection_noise.mean(), projection_n.mean() - projection_noise.mean())

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

hist = ax.plot(projection_n, label='signal+noise')
hist = ax.plot(projection_noise, label='noise')
ax.legend(loc=1)
ax.set_xlabel('Principle Component')
ax.set_ylabel('Correlation Max')
ax.set_title('Correlation Using Principle Components',pad=20)
'''
plt.tight_layout()
#plt.savefig(os.path.join(PLOTPATH, '210914_correlation_using_pc_hard'))


fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)


ax.set_xlabel('Trial')
ax.set_ylabel('Correlation Max')
ax.set_title('Correlation Using Signal, N-Trials',pad=20)


ax.plot(noise + np.flip(dataset[m, :]), label='signal+noise')
ax.plot(np.flip(dataset[m, :]), label='signal+noise')
#ax.plot(projection_noise,label='noise')

ax.legend(loc=1)

plt.tight_layout()
#plt.savefig(os.path.join(PLOTPATH, '210914_correlation_using_signal_hard'))
'''

In [ ]:

signal_noise_proj = []
noise_only_proj = []

for trial in range(25):
    m = 0
    projection_n, projection_noise, projection_signal = [], [], []

    for n in range(select_pc.shape[0]):

        noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 8192)

        noise = np.fft.fft(noise[:, 0] + 1j * noise[:, 1]) / 8192

        pc = select_pc[n, :]

        #projection_signal.append(abs(scipy.signal.fftconvolve(noise + dataset[m, :], np.flip(dataset[m, :], axis=-1))).max())
        projection_n.append(abs(scipy.signal.fftconvolve(pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), noise + np.flip(dataset[m, :], axis=-1), mode='full')).max())
        #projection_n.append(abs(scipy.signal.correlate(pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), noise + dataset[m, :], mode='full')).max())
        projection_noise.append(abs(scipy.signal.fftconvolve(noise , pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), mode='full')).max())

        #projection_n.append(abs(np.vdot(pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), noise + dataset[m, :])))
        #projection_noise.append(abs(np.vdot(noise, pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()))))

        #projection_n.append(abs(np.vdot(pc, noise + dataset[m, :])))
        #projection_noise.append(abs(np.vdot(noise, pc )))
        if n <= 10:
            projection_signal.append(abs(np.vdot(dataset[m, :], dataset[m, :])))
        if n % 1000 == 999:
            print(n+1)

    projection_n = np.array(projection_n)
    projection_noise = np.array(projection_noise)
    projection_signal = np.array(projection_signal)
    
    signal_noise_proj.append(projection_n.mean())
    noise_only_proj.append(projection_noise.mean())
    print(trial)

In [ ]:
fig = plt.figure(figsize=(8,5))
#ax = fig.add_subplot(1,1,1)

plt.hist(signal_noise_proj, label='signal+noise')
plt.hist(noise_only_proj, label='noise')

In [ ]:
signal_noise_proj = []
noise_only_proj = []

for trial in range(256):
    m = 1210
    projection_n, projection_noise, projection_signal = [], [], []

    for n in range(select_pc.shape[0]):

        noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 8192)

        noise = np.fft.fft(noise[:, 0] + 1j * noise[:, 1]) / 8192

        pc = select_pc[n, :]

        #projection_signal.append(abs(scipy.signal.fftconvolve(noise + dataset[m, :], np.flip(dataset[m, :], axis=-1))).max())
        #projection_n.append(abs(scipy.signal.fftconvolve(pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), noise + np.flip(dataset[m, :], axis=-1), mode='full')).max())
        #projection_n.append(abs(scipy.signal.correlate(pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), noise + dataset[m, :], mode='full')).max())
        projection_noise.append(abs(scipy.signal.fftconvolve(noise , pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), mode='full')).max())

        #projection_n.append(abs(np.vdot(pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), noise + dataset[m, :])))
        #projection_noise.append(abs(np.vdot(noise, pc * np.sqrt((abs(dataset[m, :]) ** 2 ).sum()))))

        #projection_n.append(abs(np.vdot(pc, noise + dataset[m, :])))
        #projection_noise.append(abs(np.vdot(noise, pc )))
        if n <= 10:
            projection_signal.append(abs(np.vdot(dataset[m, :], dataset[m, :])))
        if n % 1000 == 999:
            print(n+1)

    #projection_n = np.array(projection_n)
    projection_noise = np.array(projection_noise)
    #projection_signal = np.array(projection_signal)
    
    #signal_noise_proj.append(projection_n.mean())
    noise_only_proj.append(projection_noise.mean())
    if trial % 50 == 49:
        print(trial + 1)
    #print(trial)

In [ ]:
fig = plt.figure(figsize=(8,5))
#ax = fig.add_subplot(1,1,1)

#plt.hist(signal_noise_proj, label='signal+noise')
plt.hist(noise_only_proj, 20, label='noise')

plt.xlim(2.2e-15, 2.3e-15)

In [ ]:
pc_max = np.array(pc_max)
signal_max = np.array(signal_max)
noise_max = np.array(noise_max)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)


hist1 = ax.hist(pc_max, histtype='step')
hist2 = ax.hist(noise_max, histtype='step')
hist3 = ax.hist(signal_max, histtype='step')


In [ ]:
n = 5
m = 6

pc = evecs[:, n]
signal = dataset[m, :]

match_pc = abs(scipy.signal.correlate(pc, signal / np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), mode='same')).max()
match_sig = (abs(scipy.signal.correlate(signal / np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), signal / np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), mode='same')).max())


print(match_pc, match_sig, match_pc / match_sig)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)


ax.plot(freqs, abs(pc), label='Principle Component')
ax.plot(freqs, abs(signal / np.sqrt((abs(dataset[m, :]) ** 2 ).sum())), label='Signal')

ax.set_ylabel('Amplitude')
ax.set_xlabel('Frequency (Hz)')

ax.legend(loc=2)

plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210914_normalized_signal_and_pc'))

#print(np.vdot(pc, pc))
#print(np.vdot(signal / np.sqrt((abs(dataset[m, :]) ** 2 ).sum()), signal / np.sqrt((abs(dataset[m, :]) ** 2 ).sum())))

In [ ]:
x = scipy.signal.convolve(signal / np.sqrt((abs(signal) ** 2 ).sum()), signal / np.sqrt((abs(signal) ** 2 ).sum()), mode='same')
print(np.vdot(x, x))
plt.plot(abs(x))
#plt.plot(scipy.signal.convolve(signal / np.sqrt((abs(signal) ** 2 ).sum()), signal / np.sqrt((abs(signal) ** 2 ).sum()), mode='same').imag)

In [ ]:
x = scipy.signal.convolve(pc, signal / np.sqrt((abs(signal) ** 2 ).sum()), mode='same')
print(np.vdot(x,x))
plt.plot(abs(x))

In [ ]:
pc_match_matrix = np.zeros((dataset.shape[0], 256))

norm_dataset = dataset / np.sqrt((abs(dataset) ** 2 ).sum(axis=-1)).reshape((dataset.shape[0], 1)).repeat(dataset.shape[-1], axis=-1)

for k in range(dataset.shape[0]):
    if k % 100 == 99:
        print(k+1)
    for i in range(256):
        pc = evecs[:, i]

        x = abs(scipy.signal.correlate(pc, dataset[k, :] / np.sqrt((abs(dataset[k, :]) ** 2 ).sum()), mode='same'))

        pc_match_matrix[k, i] = x.max()

        #if i % 10 == 9:
        #    print(i + 1)



In [ ]:
np.save(os.path.join(PATH, 'damselfly/data', '210915_pc_match_matrix_normalized'), pc_match_matrix)

In [ ]:
sig_match_matrix = ((abs(dataset ) ** 2).sum(axis=-1)).reshape((dataset.shape[0], 1)).repeat(256, axis=-1)

In [ ]:
sns.set_theme(style='ticks', context='talk')
cmap = sns.color_palette('mako', as_cmap=True)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1,1,1)

img = ax.imshow((pc_match_matrix).T, aspect='auto', cmap=cmap, interpolation='none')

fig.colorbar(img, label='Match Ratio')

ax.set_xlabel('Signal Index')
ax.set_ylabel('Principle Component')

ax.set_title('Match Ratio')

plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210914_principle_components_match_ratio_matrix'))



In [ ]:
sns.set_theme(style='whitegrid', context='talk')
cmap = sns.color_palette('mako', as_cmap=True)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)



ax.plot((pc_match_matrix).mean(axis=0))


ax.set_ylabel('Mean Match')
ax.set_xlabel('Principle Component')
ax.set_title('Mean Match for Principle Component')

plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210914_mean_match_for_principle_component'))

In [ ]:
sns.set_theme(style='whitegrid', context='talk')
cmap = sns.color_palette('mako', as_cmap=True)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)



ax.plot(np.flip(np.sort((pc_match_matrix).mean(axis=0))))


ax.set_ylabel('Mean Match')
ax.set_xlabel('Principle Component')
ax.set_title('Mean Match for Principle Component, Sorted')

plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210914_mean_match_for_principle_component_sorted'))

In [ ]:
sns.set_theme(style='whitegrid', context='talk')
cmap = sns.color_palette('mako', as_cmap=True)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)


hist = ax.hist(pc_match_matrix.max(axis=-1), 32)
#ax.plot(np.mean(pc_match_matrix.max(axis=-1)) * np.ones(5492), '--' ,color='tab:red', label = f'Mean Match = {np.round(np.mean(pc_match_matrix.max(axis=-1)), 3)}')
#ax.legend(loc=(0.3, 0.6))

ax.set_ylabel('N')
ax.set_xlabel('Match Ratio')
ax.set_title('Best Principle Component Match Ratio per Signal')


plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210914_best_match_for_principle_component'))
#print(np.mean(pc_match_matrix.max(axis=-1)))

In [ ]:
covariance_real = np.matmul(signal_real_norm.T, signal_real_norm)
covariance_imag = np.matmul(signal_imag_norm.T, signal_imag_norm)
plt.figure()
plt.imshow(covariance_real, interpolation = 'none')
#plt.xlim(0, 20)
#plt.ylim(0, 20)
plt.figure()
plt.imshow(covariance_imag, interpolation = 'none')
#plt.xlim(0, 20)
#plt.ylim(0, 20)

In [ ]:
print('real')
real_evals, real_evecs = np.linalg.eig(covariance_real)
print('imag')
imag_evals, imag_evecs = np.linalg.eig(covariance_imag)

In [ ]:
print(real_evecs.shape)

In [ ]:
plt.plot(real_evals)
plt.plot(imag_evals)
plt.xlim(0, 10)

In [ ]:
plt.plot(real_evecs[1, :])
#plt.plot(real_evecs[1, :])
#plt.plot(real_evecs[2, :])
#plt.plot(real_evecs[3, :])
#plt.plot(real_evecs[4, :])
#plt.plot(real_evecs[5, :])
#plt.plot(real_evecs[6, :])
plt.xlim(8000, 8192)

# try 100 signals